In [67]:
# Dependencies and Setup
import hvplot.pandas as hvplot
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
import time
from scipy.stats import linregress
import json

# Import API key
from alan_api_keys import geoapify_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy



In [120]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../data/census_data.csv")

#Create data frame from Cenus data provided by Bryan
df = pd.DataFrame(city_data_df)

#selected columns to pull from the main df
columns_to_pull = ['NAME', 'place']

#pulled selected columns
selected_df = df[columns_to_pull]

#Split the city and state column
city_state_df = df['NAME'].str.split(',', expand=True)

#added the place column to the new city state df
city_state_df['Places'] = df['place']

#full list of cities in the new df
city_state_df.columns = ['City/Town', 'State', 'Places']

#removed the word'city' from the City/Town Column
city_state_df['City/Town'] = city_state_df['City/Town'].str.replace(' city', '', case=False)

#removed the word'town' from the City/Town Column
city_state_df['City/Town'] = city_state_df['City/Town'].str.replace(' town', '', case=False)

#removed the word'town' from the City/Town Column
city_state_df['City/Town'] = city_state_df['City/Town'].str.replace(' CDP', '', case=False)
# city_state_df['City/Town'] = city_state_df['City/Town'].str.replace(r'\s*(city|town)\s*', '', case=False)

#Reorderd the columns
state_city_df = city_state_df[['State', 'City/Town', 'Places']]

state_city_df.head()

#exported the cites file from the Census data

# state_city_df.to_csv('../data/state_cities.csv', index=False)
# state_city_df.drop(1, axis=0)
# city_state_df['City/Town'].drop_duplicates
# city_state_df.head()
# state_group_count = city_state_df.groupby(city_state_df['State']).count()
# city_state_df.sort_values(by='State')


,City/Town,State,Places
0,Auburn,Alabama,3076
1,Birmingham,Alabama,7000
2,Dothan,Alabama,21184
3,Hoover,Alabama,35896
4,Huntsville,Alabama,37000
...,...,...,...
635,Milwaukee,Wisconsin,53000
1278,Janesville,Wisconsin,37825
629,Appleton,Wisconsin,2375
1285,Cheyenne,Wyoming,13900


In [113]:
state_city_df.head()

,State,City/Town,Places
0,Alabama,Auburn,3076
1,Alabama,Birmingham,7000
2,Alabama,Dothan,21184
3,Alabama,Hoover,35896
4,Alabama,Huntsville,37000


In [117]:
radius = 1000
categories = "commercial.food_and_drink"
conditions = "health_food"
lang = "en"
limit = 20 
 

params = {
    "categories":categories,
    "limit":20,
    "city": "",
    "apiKey":geoapify_key,
    "country": "United States"    
}

# Set base URL
base_url = 'https://api.geoapify.com/v1/geocode/search'


In [119]:
for index, row in state_city_df.iterrows():
    
    # Get the city/town from the current DataFrame's row
    city = state_city_df.loc[index, "City/Town"]
    # Modify the params["categories"] based on the city if needed
    
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Handle the response
    if response.status_code == 200:
        loc_city = response.json()
        
        # Grab the first restaurant from the results and store the details in the DataFrame
        try:
            state_city_df.loc[index, "name"] = loc_city["features"][0]["properties"]["name"]
            state_city_df.loc[index, "address"] = loc_city["features"][0]["properties"]["address_line2"]
            state_city_df.loc[index, "distance"] = int(loc_city["features"][0]["properties"]["distance"])
        except (KeyError, IndexError):
            # If no restaurant is found, set the restaurant name as "No restaurant found".
            state_city_df.loc[index, "name"] = "No restaurant found"
            # Set the distance column value to np.nan to allow sorting values
            state_city_df.loc[index, "distance"] = np.nan
            
        # Log the search results
        print(f"Nearest restaurant in {city}: {state_city_df.loc[index, 'name']}")
    else:
        print(f"Error for {city}: {response.status_code}, {response.text}")

# Display sample data
state_city_df.head()

Error for Auburn: 400, {"statusCode":400,"error":"Bad Request","message":"\"city\" is not allowed to be empty"}
Error for Birmingham: 400, {"statusCode":400,"error":"Bad Request","message":"\"city\" is not allowed to be empty"}
Error for Dothan: 400, {"statusCode":400,"error":"Bad Request","message":"\"city\" is not allowed to be empty"}
Error for Hoover: 400, {"statusCode":400,"error":"Bad Request","message":"\"city\" is not allowed to be empty"}
Error for Huntsville: 400, {"statusCode":400,"error":"Bad Request","message":"\"city\" is not allowed to be empty"}
Error for Mobile: 400, {"statusCode":400,"error":"Bad Request","message":"\"city\" is not allowed to be empty"}
Error for Montgomery: 400, {"statusCode":400,"error":"Bad Request","message":"\"city\" is not allowed to be empty"}
Error for Tuscaloosa: 400, {"statusCode":400,"error":"Bad Request","message":"\"city\" is not allowed to be empty"}
Error for Anchorage municipality: 400, {"statusCode":400,"error":"Bad Request","message"

KeyboardInterrupt: 

In [118]:
# Print a message to follow up the restaurant search
print("Starting search")

# Iterate through the types_df DataFrame
for index, row in state_city_df.iterrows():
    
    # Get the ethnicity type from the current DataFrame's row
    city = state_city_df.loc[index,"City/Town"]
    # Add the current ethnicity type to the parameters
    params["categories"] = f"{categories}"
   
    # Make an API request using the params dictionary
    loc_city = requests.get(base_url, params=params).json()
    
    # # Convert the API response to JSON format
    loc_city = loc_city.json()
    
    # Grab the first restaurant from the results and store the details in the DataFrame
    try:
        state_city_df.loc[index, "name"] = loc_city["features"][0]["properties"]["name"]
        state_city_df.loc[index, "address"] = loc_city["features"][0]["properties"]["address_line2"]
        state_city_df.loc[index, "distance"] = int(loc_city["features"][0]["properties"]["distance"])
    except (KeyError, IndexError):
        # If no restaurant is found, set the restaurant name as "No restaurant found".
        state_city_df.loc[index, "name"] = "No restaurant found"
        # Set the distance column value to np.nan to allow sorting values
        state_city_df.loc[index, "distance"] = np.nan
        
    # Log the search results
    print(f"nearest {state_city_df.loc[index, 'City/Town']} restaurant: {state_city_df.loc[index, 'name']}")

# Display sample data
state_city_df.head()

Starting search


AttributeError: 'dict' object has no attribute 'json'

In [103]:
# Set the parameters for the type of place
categories = "commercial.food_and_drink"
conditions = "health_food"
radius = 5000
limit = 20
# Set the parameters for the type of search
# filters = f"circle:Newark{radius}"
# bias = f"proximity:{longitude},{latitude}"


# set up a parameters dictionary
params = {
    "categories":"commercial.food_and_drink",
    "conditions":conditions,
    "limit":20,
    "city": "newark",
    # "filter":'place:',
    # "bias":bias,
    "apiKey":geoapify_key    
}

# Set base URL
base_url = 'https://api.geoapify.com/v1/geocode/search'

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [104]:
print(response.url)

https://api.geoapify.com/v1/geocode/search?categories=commercial.food_and_drink&limit=20&city=newark&apiKey=ee2c6ace713d4289bd5d1885b1193c4c


In [102]:
# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "features": [
        {
            "bbox": [
                -74.2513334,
                40.6732502,
                -74.1141786,
                40.7881387
            ],
            "geometry": {
                "coordinates": [
                    -74.1723667,
                    40.735657
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Newark, NJ",
                "address_line2": "United States of America",
                "category": "administrative",
                "city": "Newark",
                "country": "United States",
                "country_code": "us",
                "county": "Essex County",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database License",
                    "sourcename": "openstreetmap",
                    "url": "https://www.openstreetmap.org/copyright"
            

In [114]:
radius = 5000
categories = "commercial.food_and_drink"
conditions = "health_food"
lang = "en"
limit = 20  

# Parameters dictionary
params = {
    "city": state_city_df['City/Town'],
    "categories": categories,
    "conditions": conditions,
    "apiKey": geoapify_key,
    "limit": limit,
    "lang": lang,
    "country": "United States",
}
# Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
# params["filter"] = state_city_df['City/Town']
# # params["bias"] = # YOUR CODE HERE

# # Set base URL
# base_url = 'https://api.geoapify.com/v1/geocode/search'


# # Make and API request using the params dictionaty
# name_address = requests.get(base_url, params=params)

#     # Convert the API response to JSON format
# # name_address = name_address.json()
# name_address

print("Starting health food search")

for index, row in state_city_df.iterrows():

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    # params["filter"] = state_city_df['City/Town']
    # params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = 'https://api.geoapify.com/v1/geocode/search'


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        state_city_df.loc[index, "City/Town"] = name_address["features"][0]["category"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        state_city_df.loc[index, "City/Town"] = "No hotel found"
        
    # Log the search results
    print(f"{state_city_df.loc[index, 'City']} - nearest hotel: {state_city_df.loc[index, 'category']}")

# # Display sample data
# state_city_df

Starting health food search


JSONDecodeError: Expecting value: line 1 column 1 (char 0)